# 1. Extraemos los horarios del Ciclo académico 2025 I - Plan 2018 EAP Ing. Software

Fuente [Programación de matricula SUM](https://sum.unmsm.edu.pe/alumnoWebSum/v2/matricula/programacion)
- Este dataset genera:
    1.  sum_horario_programado2025I.csv

In [35]:
import pdfplumber
import pandas as pd

## Testeamos como se imprimen las páginas

In [36]:
pdf = pdfplumber.open("..\\Data\\Origen\\Programacion_Asignaturas.pdf")
page = pdf.pages[5]
page.extract_tables()

[[['202W0901 - DESARROLLO DE TESIS\nI',
   '2.0',
   '6',
   '0A6226 - HERRERA QUISPE, JOSÉ\nALFREDO',
   '22',
   '15',
   '108',
   'MIERCOLES',
   '15:00 - 16:00'],
  [None, None, None, None, None, None, '108', 'MIERCOLES', '16:00 - 18:00'],
  ['202W0901 - DESARROLLO DE TESIS\nI',
   '2.0',
   '7',
   '0A0182 - RUIZ RIVERA, MARIA\nELENA',
   '15',
   '3',
   '--',
   'MIERCOLES',
   '19:00 - 20:00'],
  [None, None, None, None, None, None, '--', 'MIERCOLES', '20:00 - 22:00'],
  ['202W0902 - GARANTÍA DE\nSOFTWARE',
   '3.0',
   '1',
   '45993187 - LINARES JUAREZ,\nRICARDO JOSE',
   '47',
   '37',
   '109',
   'SABADO',
   '14:00 - 16:00'],
  [None, None, None, None, None, None, '109', 'SABADO', '16:00 - 18:00'],
  ['202W0902 - GARANTÍA DE\nSOFTWARE',
   '3.0',
   '2',
   '08232375 - DEL MAR ARZOLA,\nJORGE LUIS',
   '60',
   '59',
   '212',
   'SABADO',
   '14:00 - 16:00'],
  [None, None, None, None, None, None, '212', 'SABADO', '16:00 - 18:00'],
  ['202W0902 - GARANTÍA DE\nSOFTWARE',


In [37]:
pdf_path = "..\\Data\\Origen\\Programacion_Asignaturas.pdf"

In [38]:
data = []
asignatura = creditos = seccion = docente = tope = matriculados = None

# Encabezado oficial que vamos a insertar
encabezado_oficial = ['Asignatura', 'Créd.', 'Sec.', 'Docente', 'Tope', 'Matri.', 'Aula', 'Día', 'Horas Clase']

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()

        for table in tables:
            if not table or len(table) < 2:
                continue

            # Forzar encabezado solo si NO es la página 1
            if page.page_number != 1:
                table.insert(0, encabezado_oficial)

            for row in table[1:]:
                if not any(row):
                    continue
                if row[0] and "asignatura" in row[0].lower():
                    continue

                # Si contiene asignatura, es nueva fila
                if row[0]:
                    asignatura = row[0].replace('\n', ' ').strip()
                    creditos = row[1]
                    seccion = row[2]
                    docente = row[3].replace('\n', ' ').strip() if row[3] else None
                    tope = row[4]
                    matriculados = row[5]

                aula = row[6].replace('\n', ' ') if row[6] else None
                dia = row[7]
                hora = row[8]

                if any([aula, dia, hora]):
                    data.append({
                        "Asignatura": asignatura,
                        "Créditos": creditos,
                        "Sección": seccion,
                        "Docente": docente,
                        "Tope": tope,
                        "Matriculados": matriculados,
                        "Aula": aula,
                        "Día": dia,
                        "Hora": hora
                    })

In [39]:
# Convertir a DataFrame
df = pd.DataFrame(data)

# Separar hora
df[['HoraInicio', 'HoraFin']] = df['Hora'].str.extract(r'(\d{2}:\d{2})\s*-\s*(\d{2}:\d{2})')
df.drop(columns=['Hora'], inplace=True)

In [40]:
# Convertir columnas numéricas
for col in ['Créditos', 'Tope', 'Matriculados']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [41]:
# Mostrar algunos registros para validar
pd.set_option('display.max_rows', None)
print(df.head(10).to_string(index=False))

                                              Asignatura  Créditos Sección                                        Docente  Tope  Matriculados    Aula       Día HoraInicio HoraFin
                     INE002 - PROGRAMACIÓN Y COMPUTACIÓN       2.0       2              0A0182 - RUIZ RIVERA, MARIA ELENA     4             2     211   VIERNES      14:00   15:00
                     INE002 - PROGRAMACIÓN Y COMPUTACIÓN       2.0       2              0A0182 - RUIZ RIVERA, MARIA ELENA     4             2     211   VIERNES      15:00   17:00
                     INE002 - PROGRAMACIÓN Y COMPUTACIÓN       2.0       3             07240690 - DAMASO RIOS, MARIA ROSA     4             3     211    MARTES      10:00   11:00
                     INE002 - PROGRAMACIÓN Y COMPUTACIÓN       2.0       3             07240690 - DAMASO RIOS, MARIA ROSA     4             3     211    MARTES      11:00   13:00
                     INE002 - PROGRAMACIÓN Y COMPUTACIÓN       2.0       5               085774 - VERA PO

In [42]:
df["Asignatura"] = df["Asignatura"].str.strip('"')

## Normalizamos Asignaturas

In [43]:
df[['CodAsignatura', 'NomAsignatura']] = df['Asignatura'].str.split(' - ', n=1, expand=True)
df=df.drop(columns=["Asignatura"])
df.head()

,Créditos,Sección,Docente,Tope,Matriculados,Aula,Día,HoraInicio,HoraFin,CodAsignatura,NomAsignatura
0,2.0,2,"0A0182 - RUIZ RIVERA, MARIA ELENA",4,2,211,VIERNES,14:00,15:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN
1,2.0,2,"0A0182 - RUIZ RIVERA, MARIA ELENA",4,2,211,VIERNES,15:00,17:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN
2,2.0,3,"07240690 - DAMASO RIOS, MARIA ROSA",4,3,211,MARTES,10:00,11:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN
3,2.0,3,"07240690 - DAMASO RIOS, MARIA ROSA",4,3,211,MARTES,11:00,13:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN
4,2.0,5,"085774 - VERA POMALAZA, VIRGINIA",4,1,211,MIERCOLES,10:00,11:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN


## Normalizamos Docente

In [44]:
df[['CodDocente', 'NomDocente']] = df['Docente'].str.split(' - ', n=1, expand=True)
df=df.drop(columns=["Docente"])
df.head()

,Créditos,Sección,Tope,Matriculados,Aula,Día,HoraInicio,HoraFin,CodAsignatura,NomAsignatura,CodDocente,NomDocente
0,2.0,2,4,2,211,VIERNES,14:00,15:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN,0A0182,"RUIZ RIVERA, MARIA ELENA"
1,2.0,2,4,2,211,VIERNES,15:00,17:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN,0A0182,"RUIZ RIVERA, MARIA ELENA"
2,2.0,3,4,3,211,MARTES,10:00,11:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN,07240690,"DAMASO RIOS, MARIA ROSA"
3,2.0,3,4,3,211,MARTES,11:00,13:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN,07240690,"DAMASO RIOS, MARIA ROSA"
4,2.0,5,4,1,211,MIERCOLES,10:00,11:00,INE002,PROGRAMACIÓN Y COMPUTACIÓN,085774,"VERA POMALAZA, VIRGINIA"


## Dataset Final

In [46]:
df=df[["CodAsignatura","NomAsignatura","Créditos","CodDocente","NomDocente","Sección","Aula","Día","HoraInicio","HoraFin","Tope","Matriculados"]]
df.head()

,CodAsignatura,NomAsignatura,Créditos,CodDocente,NomDocente,Sección,Aula,Día,HoraInicio,HoraFin,Tope,Matriculados
0,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,0A0182,"RUIZ RIVERA, MARIA ELENA",2,211,VIERNES,14:00,15:00,4,2
1,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,0A0182,"RUIZ RIVERA, MARIA ELENA",2,211,VIERNES,15:00,17:00,4,2
2,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,07240690,"DAMASO RIOS, MARIA ROSA",3,211,MARTES,10:00,11:00,4,3
3,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,07240690,"DAMASO RIOS, MARIA ROSA",3,211,MARTES,11:00,13:00,4,3
4,INE002,PROGRAMACIÓN Y COMPUTACIÓN,2.0,085774,"VERA POMALAZA, VIRGINIA",5,211,MIERCOLES,10:00,11:00,4,1


In [47]:
df.to_csv('..\\Data\\Extraidos\\sum_horario_programado2025I.csv', index=False, sep=";",encoding='utf-8-sig')